<a href="https://colab.research.google.com/github/RemyaVKarthikeyan/Stagecoach-Project/blob/main/Timeline__Dashboard_API_LineID_StopPoint_EWT_Prediction_19_June.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DASHBOARD BUILDING - CALCULATING THE AWT, SWT AND EWT


In [ ]:
# Step 1: Get user inputs for line_id, stop_point_id, and direction
line_id = input('Enter the line id: ')
stop_point_id = input('Enter the stop point id: ')
direction = input('Enter the direction (inbound/outbound): ')
nbph = int(input('Enter the number of buses scheduled per hour: '))

Enter the line id: 22
Enter the stop point id: 490000173RF	
Enter the direction (inbound/outbound): inbound
Enter the number of buses scheduled per hour: 6


In [ ]:
# Step 2: Install required libraries
!pip install requests pandas
!pip install requests pandas plotly dash


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 8.6 MB/s eta 0:00:00


In [ ]:
# Step 3 : Import required libraries
import requests
import pandas as pd
from datetime import datetime
import time
from IPython.display import display, clear_output

In [ ]:
!pip install plotly dash
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from plotly.graph_objs import Scatter

In [ ]:
# Step 4: Define the function to fetch and display data
def fetch_arrival_predictions(line_id, stop_point_id, direction):
    base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"

    params = {'direction': direction}

    response = requests.get(base_url, params=params)
    data = response.json()

    # Step 5: Parse the data
    predictions = []
    for item in data:
        arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
        arrival_time_bst = arrival_time + pd.Timedelta(hours=1)  # Convert GMT to BST
        predictions.append({
            'Line': item['lineName'],
            'Vehicle ID': item['vehicleId'],
            'Stop Point': stop_point_id,
            'Direction': direction,
            'Expected Arrival (BST)': arrival_time_bst,
            'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')  # Format to HH:MM
        })

    # Step 6: Display the data in a readable form using pandas
    df = pd.DataFrame(predictions)

    # Sort by 'Expected Arrival (BST)'
    df = df.sort_values(by='Expected Arrival (BST)', ascending=True)

    # Convert 'Expected Arrival (HM)' to datetime format
    df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')

    # Calculate headway (considering only hour and minute)
    df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60

    # Calculate AWT (Average Waiting Time) as half of headway
    df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)

    # Calculate WAWT as the product of headway and AWT/bus
    df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)

    # Format 'Expected Arrival (BST)' column to string for better readability, ignoring seconds and microseconds
    df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))

    return df

# Step 7: Fetch and display the arrival predictions every 30 seconds
while True:
    df = fetch_arrival_predictions(line_id, stop_point_id, direction)

    # Calculate total sum of WAWT
    total_wawt = df['WAWT'].sum()

    # Calculate time difference between first and last observed bus
    min_arrival = pd.to_datetime(df['Expected Arrival (BST)']).min().replace(second=0, microsecond=0)
    max_arrival = pd.to_datetime(df['Expected Arrival (BST)']).max().replace(second=0, microsecond=0)
    time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60

    # Calculate AWT
    awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0

    # Calculate SWT
    swt = round(60 / (nbph * 2), 2)

    # Calculate EWT
    ewt = round(awt - swt, 2)

    # Create a summary DataFrame
    summary_df = pd.DataFrame({
        'Metric': ['Number of buses scheduled per hour (nbph)', 'Total WAWT (minutes)', 'Time difference between 1st and last observed buses(minutes)', 'AWT (minutes)', 'SWT (minutes)', 'EWT (minutes)'],
        'Value': [nbph, total_wawt, time_diff_minutes, awt, swt, ewt]
    })

    clear_output(wait=True)  # Clear previous output to update the table
    print("Arrival Predictions:")
    display(df[['Line', 'Vehicle ID', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']])
    print("Summary Metrics:")
    display(summary_df)

    time.sleep(30)  # Wait for 30 seconds before the next update

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

FIRST DYNAMIC DASHBOARD

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, clear_output

# Function to create the dynamic timeline dashboard
def create_dashboard(df):
    current_time = datetime.now().strftime('%H:%M:%S')

    # Create a timeline figure
    fig = go.Figure()

    # Add a big square for the stop point at the right end
    fig.add_trace(go.Scatter(
        x=[1],
        y=[0],
        mode='markers',
        marker=dict(size=20, symbol='square', color='black'),
        name='Stop Point'
    ))

    # Add circles for each bus moving towards the stop point
    for i, row in df.iterrows():
        fig.add_trace(go.Scatter(
            x=[1 - (datetime.strptime(row['Expected Arrival (BST)'], '%H:%M:%S') - datetime.strptime(current_time, '%H:%M:%S')).total_seconds() / (30 * 60)],
            y=[i],
            mode='markers+text',
            marker=dict(size=12, color=px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)]),
            text=row['Vehicle ID'],
            textposition='top center',
            name=row['Vehicle ID']
        ))

    # Update layout
    fig.update_layout(
        title='Bus Positions by Expected Arrival Time',
        xaxis=dict(title='Timeline', showticklabels=False),
        yaxis=dict(title='Vehicle IDs', showticklabels=False),
        showlegend=False
    )

    return fig

# Fetch and display the arrival predictions every 30 seconds
while True:
    df = fetch_arrival_predictions(line_id, stop_point_id, direction)

    # Calculate total sum of WAWT
    total_wawt = df['WAWT'].sum()

    # Calculate time difference between first and last observed bus
    min_arrival = pd.to_datetime(df['Expected Arrival (BST)']).min().replace(second=0, microsecond=0)
    max_arrival = pd.to_datetime(df['Expected Arrival (BST)']).max().replace(second=0, microsecond=0)
    time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60

    # Calculate AWT
    awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0

    # Calculate SWT
    swt = round(60 / (nbph * 2), 2)

    # Calculate EWT
    ewt = round(awt - swt, 2)

    # Create a summary DataFrame
    summary_df = pd.DataFrame({
        'Metric': ['Number of buses scheduled per hour (nbph)', 'Total WAWT (minutes)', 'Time difference between 1st and last observed buses(minutes)', 'AWT (minutes)', 'SWT (minutes)', 'EWT (minutes)'],
        'Value': [nbph, total_wawt, time_diff_minutes, awt, swt, ewt]
    })

    clear_output(wait=True)  # Clear previous output to update the table

    # Display the updated predictions and summary metrics
    print("Arrival Predictions:")
    display(df[['Line', 'Vehicle ID', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']])
    print("Summary Metrics:")
    display(summary_df)

    # Create and display the dynamic timeline dashboard
    fig = create_dashboard(df)
    fig.show()

    time.sleep(30)  # Wait for 30 seconds before the next update


Arrival Predictions:


,Line,Vehicle ID,Stop Point,Direction,Expected Arrival (BST),Headway (minutes),AWT/bus (minutes),WAWT
0,22,BV66VHJ,490000173RF,inbound,11:41:00,0.0,0.0,0.0
1,22,LJ62KGF,490000173RF,inbound,11:51:00,10.0,5.0,50.0
2,22,YY66OZE,490000173RF,inbound,12:02:00,11.0,5.5,60.5


Summary Metrics:


,Metric,Value
0,Number of buses scheduled per hour (nbph),6.00
1,Total WAWT (minutes),110.50
2,Time difference between 1st and last observed ...,21.00
3,AWT (minutes),5.26
4,SWT (minutes),5.00
5,EWT (minutes),0.26


KeyboardInterrupt: 

Hover over data changed to the expected arrival time to the stop point

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, clear_output
from datetime import datetime
import pandas as pd

# Function to create the dynamic timeline dashboard
def create_dashboard(df):
    current_time = datetime.now().strftime('%H:%M:%S')

    # Create a timeline figure
    fig = go.Figure()

    # Add a big square for the stop point at the right end
    fig.add_trace(go.Scatter(
        x=[1],
        y=[0],
        mode='markers',
        marker=dict(size=20, symbol='square', color='black'),
        name='Stop Point',
        hoverinfo='skip'  # Skip hover info for stop point
    ))

    # Add circles for each bus moving towards the stop point
    for i, row in df.iterrows():
        expected_arrival = row['Expected Arrival (BST)']
        fig.add_trace(go.Scatter(
            x=[1 - (datetime.strptime(expected_arrival, '%H:%M:%S') - datetime.strptime(current_time, '%H:%M:%S')).total_seconds() / (30 * 60)],
            y=[i],
            mode='markers+text',
            marker=dict(size=12, color=px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)]),
            text=row['Vehicle ID'],
            textposition='top center',
            name=row['Vehicle ID'],
            hovertext=f'Expected Arrival: {expected_arrival}',
            hoverinfo='text'
        ))

    # Update layout
    fig.update_layout(
        title='Bus Positions by Expected Arrival Time',
        xaxis=dict(title='Timeline', showticklabels=False),
        yaxis=dict(title='Vehicle IDs', showticklabels=False),
        showlegend=False
    )

    return fig

# Fetch and display the arrival predictions every 30 seconds
while True:
    df = fetch_arrival_predictions(line_id, stop_point_id, direction)

    # Calculate total sum of WAWT
    total_wawt = df['WAWT'].sum()

    # Calculate time difference between first and last observed bus
    min_arrival = pd.to_datetime(df['Expected Arrival (BST)']).min().replace(second=0, microsecond=0)
    max_arrival = pd.to_datetime(df['Expected Arrival (BST)']).max().replace(second=0, microsecond=0)
    time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60

    # Calculate AWT
    awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0

    # Calculate SWT
    swt = round(60 / (nbph * 2), 2)

    # Calculate EWT
    ewt = round(awt - swt, 2)

    # Create a summary DataFrame
    summary_df = pd.DataFrame({
        'Metric': ['Number of buses scheduled per hour (nbph)', 'Total WAWT (minutes)', 'Time difference between 1st and last observed buses(minutes)', 'AWT (minutes)', 'SWT (minutes)', 'EWT (minutes)'],
        'Value': [nbph, total_wawt, time_diff_minutes, awt, swt, ewt]
    })

    clear_output(wait=True)  # Clear previous output to update the table

    # Display the updated predictions and summary metrics
    print("Arrival Predictions:")
    display(df[['Line', 'Vehicle ID', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']])
    print("Summary Metrics:")
    display(summary_df)

    # Create and display the dynamic timeline dashboard
    fig = create_dashboard(df)
    fig.show()

    time.sleep(30)  # Wait for 30 seconds before the next update


Arrival Predictions:


,Line,Vehicle ID,Stop Point,Direction,Expected Arrival (BST),Headway (minutes),AWT/bus (minutes),WAWT
1,22,BV66VJP,490000173RF,inbound,12:30:00,0.0,0.0,0.0
3,22,YY66OYU,490000173RF,inbound,12:35:00,5.0,2.5,12.5
0,22,BV66VHK,490000173RF,inbound,12:46:00,11.0,5.5,60.5
2,22,BV66VLE,490000173RF,inbound,12:57:00,11.0,5.5,60.5


Summary Metrics:


,Metric,Value
0,Number of buses scheduled per hour (nbph),6.00
1,Total WAWT (minutes),133.50
2,Time difference between 1st and last observed ...,27.00
3,AWT (minutes),4.94
4,SWT (minutes),5.00
5,EWT (minutes),-0.06


KeyboardInterrupt: 

Hover over data changed to the expected arrival time to the stop point and Stop point label included

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, clear_output
from datetime import datetime
import pandas as pd

# Function to create the dynamic timeline dashboard
def create_dashboard(df):
    current_time = datetime.now().strftime('%H:%M:%S')

    # Create a timeline figure
    fig = go.Figure()

    # Add a big square for the stop point at the right end
    fig.add_trace(go.Scatter(
        x=[1],
        y=[0],
        mode='markers',
        marker=dict(size=20, symbol='square', color='black'),
        name='Stop Point',
        hoverinfo='skip'  # Skip hover info for stop point
    ))

    # Add label for the stop point
    fig.add_annotation(
        x=1,
        y=0.2,
        text=df['Stop Point'].iloc[0],  # Display stop point value
        showarrow=False,
        font=dict(size=12, color='black'),
        align='center'
    )

    # Add circles for each bus moving towards the stop point
    for i, row in df.iterrows():
        expected_arrival = row['Expected Arrival (BST)']
        fig.add_trace(go.Scatter(
            x=[1 - (datetime.strptime(expected_arrival, '%H:%M:%S') - datetime.strptime(current_time, '%H:%M:%S')).total_seconds() / (30 * 60)],
            y=[i],
            mode='markers+text',
            marker=dict(size=12, color=px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)]),
            text=row['Vehicle ID'],
            textposition='top center',
            name=row['Vehicle ID'],
            hovertext=f'Expected Arrival: {expected_arrival}',
            hoverinfo='text'
        ))

    # Update layout
    fig.update_layout(
        title='Bus Positions by Expected Arrival Time',
        xaxis=dict(title='Timeline', showticklabels=False),
        yaxis=dict(title='Vehicle IDs', showticklabels=False),
        showlegend=False
    )

    return fig

# Fetch and display the arrival predictions every 30 seconds
while True:
    df = fetch_arrival_predictions(line_id, stop_point_id, direction)

    # Calculate total sum of WAWT
    total_wawt = df['WAWT'].sum()

    # Calculate time difference between first and last observed bus
    min_arrival = pd.to_datetime(df['Expected Arrival (BST)']).min().replace(second=0, microsecond=0)
    max_arrival = pd.to_datetime(df['Expected Arrival (BST)']).max().replace(second=0, microsecond=0)
    time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60

    # Calculate AWT
    awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0

    # Calculate SWT
    swt = round(60 / (nbph * 2), 2)

    # Calculate EWT
    ewt = round(awt - swt, 2)

    # Create a summary DataFrame
    summary_df = pd.DataFrame({
        'Metric': ['Number of buses scheduled per hour (nbph)', 'Total WAWT (minutes)', 'Time difference between 1st and last observed buses(minutes)', 'AWT (minutes)', 'SWT (minutes)', 'EWT (minutes)'],
        'Value': [nbph, total_wawt, time_diff_minutes, awt, swt, ewt]
    })

    clear_output(wait=True)  # Clear previous output to update the table

    # Display the updated predictions and summary metrics
    print("Arrival Predictions:")
    display(df[['Line', 'Vehicle ID', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']])
    print("Summary Metrics:")
    display(summary_df)

    # Create and display the dynamic timeline dashboard
    fig = create_dashboard(df)
    fig.show()

    time.sleep(15)  # Wait for 30 seconds before the next update





Arrival Predictions:


,Line,Vehicle ID,Stop Point,Direction,Expected Arrival (BST),Headway (minutes),AWT/bus (minutes),WAWT
3,22,YY66OYU,490000173RF,inbound,12:44:00,0.0,0.0,0.0
1,22,BV66VHK,490000173RF,inbound,12:46:00,2.0,1.0,2.0
2,22,BV66VLE,490000173RF,inbound,12:57:00,11.0,5.5,60.5
0,22,BT66MSO,490000173RF,inbound,13:07:00,10.0,5.0,50.0


Summary Metrics:


,Metric,Value
0,Number of buses scheduled per hour (nbph),6.00
1,Total WAWT (minutes),112.50
2,Time difference between 1st and last observed ...,23.00
3,AWT (minutes),4.89
4,SWT (minutes),5.00
5,EWT (minutes),-0.11


KeyboardInterrupt: 

Designing a new dashboard with different display

Modifying the code for a time line based dashboard

Dashboard- Time line based

Modifying to remove the time stamp on the x axis and including the current time (BST) above the stop point

In [ ]:
import plotly.graph_objects as go
from IPython.display import display, clear_output
from datetime import datetime
import pandas as pd
import time
import pytz  # Import pytz for timezone handling

# Function to convert time to BST
def convert_to_bst(dt):
    # Define UTC and BST timezones
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')  # BST timezone for London

    # Convert input datetime to UTC timezone
    utc_dt = utc_tz.localize(dt)

    # Convert UTC time to BST
    bst_dt = utc_dt.astimezone(bst_tz)

    return bst_dt.strftime('%H:%M:%S')

# Function to create the dynamic timeline dashboard
def create_dashboard(df):
    current_time_utc = datetime.utcnow()  # Get current UTC time
    current_time_bst = convert_to_bst(current_time_utc)  # Convert to BST

    # Create a timeline figure
    fig = go.Figure()

    # Add a big square for the stop point at the right end
    fig.add_trace(go.Scatter(
        x=[current_time_bst],
        y=[0],
        mode='markers',
        marker=dict(size=40, symbol='square', color='black'),
        name='Stop Point',
        text=f"Stop Point: {df['Stop Point'].iloc[0]}<br>Current Time (BST): {current_time_bst}",  # Display stop point value and current BST time
        hoverinfo='text'
    ))

    # Add label for the current BST time above the square
    fig.add_annotation(
        x=current_time_bst,
        y=0,
        text=f"{df['Stop Point'].iloc[0]}",
        showarrow=False,
        font=dict(size=14, color="black"),
        yshift=30  # Adjust the yshift to move the label above the square
    )

    # Add circles for each bus moving towards the stop point
    for i, row in df.iterrows():
        expected_arrival = row['Expected Arrival (BST)']
        fig.add_trace(go.Scatter(
            x=[expected_arrival],
            y=[0],  # Y position of the stop point (fixed at 0)
            mode='markers+text',
            marker=dict(size=12, color=px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)]),
            text=row['Vehicle ID'],
            textposition='top center',
            name=row['Vehicle ID'],
            hovertext=f'Expected Arrival: {expected_arrival}',
            hoverinfo='text'
        ))

    # Update layout
    fig.update_layout(
        title='Bus Positions by Expected Arrival Time',
        xaxis=dict(title='', showticklabels=False),  # Remove x-axis tick labels and title
        yaxis=dict(showticklabels=False),  # Remove y-axis tick labels
        showlegend=True
    )

    return fig

# Fetch and display the arrival predictions every 15 seconds
while True:
    df = fetch_arrival_predictions(line_id, stop_point_id, direction)

    clear_output(wait=True)  # Clear previous output to update the table

    # Display the updated predictions
    print("Arrival Predictions:")
    display(df[['Line', 'Vehicle ID', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']])

    # Create and display the dynamic timeline dashboard
    fig = create_dashboard(df)
    fig.show()

    time.sleep(15)  # Wait for 15 seconds before the next update


NameError: name 'fetch_arrival_predictions' is not defined

In [ ]:
import plotly.graph_objects as go
from IPython.display import display, clear_output
from datetime import datetime
import pandas as pd
import time
import pytz  # Import pytz for timezone handling

# Function to convert time to BST
def convert_to_bst(dt):
    # Define UTC and BST timezones
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')  # BST timezone for London

    # Convert input datetime to UTC timezone
    utc_dt = utc_tz.localize(dt)

    # Convert UTC time to BST
    bst_dt = utc_dt.astimezone(bst_tz)

    return bst_dt.strftime('%H:%M:%S')

# Function to create the dynamic timeline dashboard
def create_dashboard(df):
    current_time_utc = datetime.utcnow()  # Get current UTC time
    current_time_bst = convert_to_bst(current_time_utc)  # Convert to BST

    # Create a timeline figure
    fig = go.Figure()

    # Add a big square for the stop point at the right end
    fig.add_trace(go.Scatter(
        x=[current_time_bst],
        y=[0],
        mode='markers',
        marker=dict(size=40, symbol='square', color='black'),
        name='Stop Point',
        text=df['Stop Point'].iloc[0],  # Display stop point value
        hoverinfo='text'
    ))

    # Add label for the stop point value above the square
    fig.add_annotation(
        x=current_time_bst,
        y=0,
        text=f"{df['Stop Point'].iloc[0]}",
        showarrow=False,
        font=dict(size=14, color="black"),
        yshift=30  # Adjust the yshift to move the label above the square
    )

    # Add circles for each bus moving towards the stop point
    for i, row in df.iterrows():
        expected_arrival = row['Expected Arrival (BST)']
        fig.add_trace(go.Scatter(
            x=[expected_arrival],
            y=[0],  # Y position of the stop point (fixed at 0)
            mode='markers+text',
            marker=dict(size=12, color=px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)]),
            text=row['Vehicle ID'],
            textposition='top center',
            name=row['Vehicle ID'],
            hovertext=f'Expected Arrival: {expected_arrival}',
            hoverinfo='text'
        ))

    # Update layout
    fig.update_layout(
        title='Bus Positions by Expected Arrival Time',
        xaxis=dict(title='Time (BST)'),
        yaxis=dict(showticklabels=False),  # Remove y-axis tick labels
        showlegend=True
    )

    return fig

# Fetch and display the arrival predictions every 15 seconds
while True:
    df = fetch_arrival_predictions(line_id, stop_point_id, direction)

    clear_output(wait=True)  # Clear previous output to update the table

    # Display the updated predictions
    print("Arrival Predictions:")
    display(df[['Line', 'Vehicle ID', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']])

    # Create and display the dynamic timeline dashboard
    fig = create_dashboard(df)
    fig.show()

    time.sleep(15)  # Wait for 15 seconds before the next update


Arrival Predictions:


,Line,Vehicle ID,Stop Point,Direction,Expected Arrival (BST),Headway (minutes),AWT/bus (minutes),WAWT
0,22,BT66MSO,490000173RF,inbound,13:07:00,0.0,0.0,0.0
2,22,BV66VLP,490000173RF,inbound,13:18:00,11.0,5.5,60.5
1,22,BV66VHH,490000173RF,inbound,13:29:00,11.0,5.5,60.5


KeyboardInterrupt: 

In [ ]:
i am just testing